In [1]:
import dataclasses
import pickle

import pandas as pd
from tqdm.auto import tqdm  # for notebooks

from src.app.core.api import Features
from src.app.core.model import AdvancedModel

# (добавил, чтобы мр прошёл)

tqdm.pandas()

In [2]:
data = pd.read_csv('test_X.csv')
data.set_index('SK_ID_CURR', inplace=True)

Я считаю порог _thresold = 0.3 в Simple model слишком большим (объяснение ниже)

Параметры формулы для объяснения, которое последует позже

sum - сколько даём
d - вероятность дефолта (этот параметр ограничиваем _threshold)
n - человек с таким дефолтом
percent - процент под который даём

Мне кажется, что порог очень высокий так как формула,
чтобы банк вышел в ноль по прибыли следующая
(-sum*d*n) + (sum*(n-d*n)*percent = 0

После оптимизации параметров получаем формулу
percent/(1+percent) - d = 0

Следовательно при d = 0.3
получаем процент переплаты 42.85% по кредиту,
чтобы выйти в ноль для банка для такого порога
по мне это высокий порог, хотя зависит от того насколько берут кредит

Я бы сделал порог динамическим и вычислял порог в калькуляторе,
чтобы более точно регулировать кому давать кредит, а кому не давать кредит.

Вычислял бы порог в зависимости от
1. инфляции,
2. заработка,
3. количества лет на текущей работе,
4. процента по кредиту,
5. процента ЦБ,
6. наличие дома,
6. и т.д.

Не уверен, что в данном задании это можно делать, поэтому не делал

In [3]:
model = AdvancedModel('./catboost_model.pkl')
with open('./catboost_model.pkl', 'rb') as pickled_model:
    catboost = pickle.load(pickled_model)

Проверяем одинаково ли предсказывают модели

In [4]:
model._predict_proba(Features(*data.iloc[0]))

0.046397480553594675

catboost предсказание

In [5]:
catboost.predict_proba(data.iloc[0])[1]

0.046397480553594675

Как видно предсказывает модель обёрнутая и не обёрнутая одинаково, так как был возвращён порядок строчек

Модель всё предсказала верно, но может быть не идентичная конвертация из-за проблем с неймингом в датафрейме, проверим

In [6]:
print(all(
    pd.DataFrame([dataclasses.asdict(Features(*data.iloc[0]))])[list(Features.__annotations__)] ==
    data.iloc[0]))

True


C:\Users\user\AppData\Local\Temp\ipykernel_7348\1701032786.py:2: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  pd.DataFrame([dataclasses.asdict(Features(*data.iloc[0]))])[list(Features.__annotations__)] ==


Всё правильно конвертировали получается

Теперь можем предсказывать остальные значения, поэлеметное предсказание очень долгое (>40 минут), поэтому применим только к 500 значениям

In [7]:
small_data = data.iloc[:500]

In [8]:
catboost_preds = catboost.predict_proba(small_data)[:, 1]

In [9]:
small_data['result'] = small_data.progress_apply(
    lambda x: model.get_scoring_result(
        Features(*x)
    ),
    axis=1
)

  0%|          | 0/500 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_7348\2720292562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_data['result'] = small_data.progress_apply(


In [10]:
small_data['decision'] = small_data['result'].apply(lambda x: x.decision.name)
small_data['amount'] = small_data['result'].apply(lambda x: x.amount)
small_data['proba'] = small_data['result'].apply(lambda x: x.proba)

C:\Users\user\AppData\Local\Temp\ipykernel_7348\592975944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_data['decision'] = small_data['result'].apply(lambda x: x.decision.name)
C:\Users\user\AppData\Local\Temp\ipykernel_7348\592975944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_data['amount'] = small_data['result'].apply(lambda x: x.amount)
C:\Users\user\AppData\Local\Temp\ipykernel_7348\592975944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [11]:
pd.concat([small_data.reset_index(),
           pd.DataFrame(catboost_preds, columns=['catboost_preds'])],
          axis=1)[['SK_ID_CURR',
                   'result',
                   'decision',
                   'amount',
                   'proba',
                   'catboost_preds']]

,SK_ID_CURR,result,decision,amount,proba,catboost_preds
0,100001,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,622080.0,0.046397,0.046397
1,100005,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,356400.0,0.122031,0.122031
2,100013,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,414720.0,0.038941,0.038941
3,100028,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,2721600.0,0.070644,0.070644
4,100038,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,648000.0,0.111016,0.111016
...,...,...,...,...,...,...
495,103462,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,207360.0,0.025894,0.025894
496,103473,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,777600.0,0.062936,0.062936
497,103474,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,907200.0,0.040137,0.040137
498,103475,ScoringResult(decision=<ScoringDecision.ACCEPT...,ACCEPTED,216000.0,0.110047,0.110047


Для кеггла (ОСТОРОЖНО более 40 минут выполняется)

In [12]:
data['TARGET'] = data.apply(
    lambda x: model._predict_proba(Features(*x)), axis=1)
data.reset_index(inplace=True)
data = data[['SK_ID_CURR', 'TARGET']]
data.to_csv('./test_preds.csv', index=False)
print('Good')

Good
